# Cross-Geometry Prediction: Square Cylinder using Circular Model

This notebook demonstrates an **experimental cross-geometry prediction**:
1. **LBM Simulation** (C++) for square cylinder at Re=35
2. **Use trained circular cylinder model** to predict square cylinder
3. **Compare** with ground truth to show geometry mismatch effects

## Warning
This is an educational demonstration. The model was trained on **circular** cylinders,
so predictions for **square** cylinders will have significant errors.


## 1. Installation

### Install DeepXDE


In [ ]:
# Install DeepXDE and dependencies
!pip install deepxde
!pip install matplotlib numpy

### Verify Installation

In [ ]:
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import time

print(f"DeepXDE version: {dde.__version__}")
print(f"Backend: {dde.backend.backend_name}")

## 2. Install C++ Compiler (For Google Colab)

Google Colab comes with g++ pre-installed, but let's verify and ensure we have the right version.


In [ ]:
# Check if g++ is installed
!g++ --version

# If not installed (unlikely in Colab), install it:
# !apt-get update
# !apt-get install -y g++

## 3. LBM Simulation for Square Cylinder (C++)

### Complete C++ Code

We'll create the full LBM simulation code for square cylinder.


In [ ]:
%%writefile lbm_square_cylinder.cpp
#include <iostream>
#include <fstream>
#include <cmath>
#include <vector>
#include <iomanip>

using namespace std;

// D2Q9 Lattice parameters
const int Q = 9;
const int ex[Q] = {0, 1, 0, -1, 0, 1, -1, -1, 1};
const int ey[Q] = {0, 0, 1, 0, -1, 1, 1, -1, -1};
const double w[Q] = {4.0/9, 1.0/9, 1.0/9, 1.0/9, 1.0/9, 1.0/36, 1.0/36, 1.0/36, 1.0/36};

// Domain parameters
const int NX = 400;
const int NY = 100;
const double SQUARE_X = 100.0;
const double SQUARE_Y = 50.0;
const double SQUARE_SIDE = 20.0;

// LBM parameters
const double U_LID = 0.1;
const double TOL = 1e-5;
const int MAX_ITER = 50000;
const int PRINT_FREQ = 500;

// Function to check if point is inside square
bool isInsideSquare(int x, int y) {
    double dx = abs(x - SQUARE_X);
    double dy = abs(y - SQUARE_Y);
    double half_side = SQUARE_SIDE / 2.0;
    return (dx <= half_side && dy <= half_side);
}

// Equilibrium distribution function
double feq(int k, double rho, double ux, double uy) {
    double cu = ex[k] * ux + ey[k] * uy;
    double u2 = ux * ux + uy * uy;
    return w[k] * rho * (1.0 + 3.0 * cu + 4.5 * cu * cu - 1.5 * u2);
}

void simulate(double reynolds) {
    double nu = U_LID * SQUARE_SIDE / reynolds;
    double tau = 3.0 * nu + 0.5;
    double omega = 1.0 / tau;
    
    cout << "\\n=== Simulating Square Cylinder Re = " << reynolds << " ===" << endl;
    cout << "Domain: " << NX << " x " << NY << endl;
    cout << "nu = " << nu << ", tau = " << tau << ", omega = " << omega << endl;
    
    // Initialize arrays
    vector<vector<vector<double>>> f(NX, vector<vector<double>>(NY, vector<double>(Q)));
    vector<vector<vector<double>>> f_new(NX, vector<vector<double>>(NY, vector<double>(Q)));
    vector<vector<double>> rho(NX, vector<double>(NY, 1.0));
    vector<vector<double>> ux(NX, vector<double>(NY, 0.0));
    vector<vector<double>> uy(NX, vector<double>(NY, 0.0));
    vector<vector<bool>> obstacle(NX, vector<bool>(NY, false));
    
    // Mark obstacle cells
    for (int x = 0; x < NX; x++) {
        for (int y = 0; y < NY; y++) {
            obstacle[x][y] = isInsideSquare(x, y);
        }
    }
    
    // Initialize distribution functions
    for (int x = 0; x < NX; x++) {
        for (int y = 0; y < NY; y++) {
            if (!obstacle[x][y]) {
                ux[x][y] = U_LID;
                for (int k = 0; k < Q; k++) {
                    f[x][y][k] = feq(k, 1.0, ux[x][y], 0.0);
                }
            }
        }
    }
    
    // Main iteration loop
    for (int iter = 0; iter < MAX_ITER; iter++) {
        // Collision
        for (int x = 0; x < NX; x++) {
            for (int y = 0; y < NY; y++) {
                if (obstacle[x][y]) continue;
                for (int k = 0; k < Q; k++) {
                    double f_eq = feq(k, rho[x][y], ux[x][y], uy[x][y]);
                    f_new[x][y][k] = f[x][y][k] - omega * (f[x][y][k] - f_eq);
                }
            }
        }
        
        // Streaming with boundary conditions
        for (int x = 0; x < NX; x++) {
            for (int y = 0; y < NY; y++) {
                if (obstacle[x][y]) continue;
                for (int k = 0; k < Q; k++) {
                    int x_prev = x - ex[k];
                    int y_prev = y - ey[k];
                    
                    if (x_prev < 0) {
                        f[x][y][k] = feq(k, 1.0, U_LID, 0.0);
                    } else if (x_prev >= NX) {
                        f[x][y][k] = f[NX-1][y][k];
                    } else if (y_prev < 0 || y_prev >= NY || obstacle[x_prev][y_prev]) {
                        int k_opp = 0;
                        for (int kk = 0; kk < Q; kk++) {
                            if (ex[kk] == -ex[k] && ey[kk] == -ey[k]) {
                                k_opp = kk;
                                break;
                            }
                        }
                        f[x][y][k] = f_new[x][y][k_opp];
                    } else {
                        f[x][y][k] = f_new[x_prev][y_prev][k];
                    }
                }
            }
        }
        
        // Compute macroscopic quantities
        double max_diff = 0.0;
        for (int x = 0; x < NX; x++) {
            for (int y = 0; y < NY; y++) {
                if (obstacle[x][y]) continue;
                double rho_new = 0.0, ux_new = 0.0, uy_new = 0.0;
                for (int k = 0; k < Q; k++) {
                    rho_new += f[x][y][k];
                    ux_new += ex[k] * f[x][y][k];
                    uy_new += ey[k] * f[x][y][k];
                }
                ux_new /= rho_new;
                uy_new /= rho_new;
                double diff = abs(ux_new - ux[x][y]) + abs(uy_new - uy[x][y]);
                if (diff > max_diff) max_diff = diff;
                rho[x][y] = rho_new;
                ux[x][y] = ux_new;
                uy[x][y] = uy_new;
            }
        }
        
        if (iter % PRINT_FREQ == 0) {
            cout << "  Iter: " << iter << ", Max Diff: " << scientific << setprecision(6) << max_diff << endl;
        }
        
        if (max_diff < TOL) {
            cout << "  Converged at iteration " << iter << endl;
            break;
        }
    }
    
    // Save results
    string filename = "square_cylinder_re35.txt";
    ofstream outfile(filename);
    outfile << "# x y u v rho" << endl;
    for (int y = 0; y < NY; y++) {
        for (int x = 0; x < NX; x++) {
            if (!obstacle[x][y]) {
                outfile << x << " " << y << " " 
                       << scientific << setprecision(8) 
                       << ux[x][y] << " " << uy[x][y] << " " << rho[x][y] << endl;
            }
        }
    }
    outfile.close();
    cout << "  Results saved to " << filename << endl;
}

int main() {
    simulate(35.0);
    return 0;
}


### Compile and Run LBM Simulation

In [ ]:
# Compile the C++ code
!g++ -O3 -std=c++11 lbm_square_cylinder.cpp -o lbm_square_cylinder

# Run the simulation
!./lbm_square_cylinder

print("\nLBM simulation completed! Generated: square_cylinder_re35.txt")

## 4. Load Circular Cylinder Training Data

We'll use the circular cylinder model trained on Re=10, 20, 30.


In [ ]:
# Configuration
CYLINDER_X = 100.0 / 400.0
CYLINDER_Y = 50.0 / 100.0
SQUARE_SIDE = 20.0 / 400.0
U_LID = 0.1

HIDDEN_LAYERS = 8
NEURONS_PER_LAYER = 100
EPOCHS = 1000  # Quick training for demonstration

def load_lbm_data(re_values):
    """Load LBM data for circular cylinders."""
    all_data = []
    
    for re in re_values:
        filename = f"circular_cylinder_re{int(re)}.txt"
        print(f"Loading {filename}...")
        
        try:
            data = np.loadtxt(filename, comments='#')
            
            x_norm = data[:, 0] / 400.0
            y_norm = data[:, 1] / 100.0
            u_norm = data[:, 2] / U_LID
            v_norm = data[:, 3] / U_LID
            p_norm = (data[:, 4] - 1.0) * 3.0
            
            stride = 5
            indices = np.arange(0, len(x_norm), stride)
            
            all_data.append({
                'x': x_norm[indices],
                'y': y_norm[indices],
                'u': u_norm[indices],
                'v': v_norm[indices],
                'p': p_norm[indices],
                're': re,
                'n_points': len(indices)
            })
            
            print(f"  Loaded {len(indices)} points for Re={re}")
        except FileNotFoundError:
            print(f"  Warning: {filename} not found!")
    
    return all_data

def prepare_training_data(lbm_data):
    """Prepare training data."""
    x_list, y_list, re_list = [], [], []
    u_list, v_list, p_list = [], [], []
    
    for data in lbm_data:
        n = data['n_points']
        x_list.append(data['x'])
        y_list.append(data['y'])
        u_list.append(data['u'])
        v_list.append(data['v'])
        p_list.append(data['p'])
        re_list.append(np.full(n, data['re'] / 40.0))
    
    X_data = np.column_stack([
        np.concatenate(x_list),
        np.concatenate(y_list),
        np.concatenate(re_list)
    ]).astype(np.float32)
    
    y_data = np.column_stack([
        np.concatenate(u_list),
        np.concatenate(v_list),
        np.concatenate(p_list)
    ]).astype(np.float32)
    
    return X_data, y_data

# Load circular cylinder data
training_re = [10, 20, 30]
lbm_data = load_lbm_data(training_re)
X_train, y_train = prepare_training_data(lbm_data)

print(f"\nTotal training points: {len(X_train)}")
print("NOTE: Training data is from CIRCULAR cylinders!")

## 4. Train Model on Circular Cylinder Data

In [ ]:
# Create dataset
data = dde.data.Triple(X_train, y_train, X_train, y_train)

# Neural network
layer_sizes = [3] + [NEURONS_PER_LAYER] * HIDDEN_LAYERS + [3]
net = dde.nn.FNN(layer_sizes, "tanh", "Glorot uniform")

# Create and compile model
model = dde.Model(data, net)
model.compile("adam", lr=1e-3)

# Quick training
print("\nTraining on circular cylinder data...")
start_time = time.time()

losshistory, train_state = model.train(
    iterations=EPOCHS,
    display_every=200
)

training_time = time.time() - start_time
print(f"\nTraining completed in {training_time:.2f} seconds")

## 5. Predict Square Cylinder at Re=35

Now we use the circular cylinder model to predict square cylinder flow.
This is where the geometry mismatch will cause errors!


In [ ]:
def is_inside_square(x, y):
    """Check if point is inside square."""
    dx = abs(x - CYLINDER_X)
    dy = abs(y - CYLINDER_Y)
    half_side = SQUARE_SIDE / 2.0
    return (dx <= half_side and dy <= half_side)

# Create prediction grid
nx, ny = 400, 100
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
X_grid, Y_grid = np.meshgrid(x, y)

x_flat = X_grid.flatten()
y_flat = Y_grid.flatten()
re_flat = np.full_like(x_flat, 35.0 / 40.0)

X_test = np.column_stack([x_flat, y_flat, re_flat])

# Predict using circular model
print("Predicting SQUARE cylinder using CIRCULAR model...")
print("(Expect geometry mismatch errors!)")
y_pred = model.predict(X_test)

U_pred = y_pred[:, 0].reshape(ny, nx) * U_LID
V_pred = y_pred[:, 1].reshape(ny, nx) * U_LID
P_pred = y_pred[:, 2].reshape(ny, nx)

# Mask square region
for i in range(ny):
    for j in range(nx):
        if is_inside_square(X_grid[i, j], Y_grid[i, j]):
            U_pred[i, j] = np.nan
            V_pred[i, j] = np.nan
            P_pred[i, j] = np.nan

print("Prediction complete!")

## 6. Load Square Cylinder Ground Truth

In [ ]:
# Load LBM ground truth for square cylinder
data_gt = np.loadtxt("square_cylinder_re35.txt", comments='#')

x_gt = data_gt[:, 0]
y_gt = data_gt[:, 1]
u_gt = data_gt[:, 2]
v_gt = data_gt[:, 3]

# Create grid
U_GT = np.full((ny, nx), np.nan)
V_GT = np.full((ny, nx), np.nan)

for i in range(len(x_gt)):
    xi = int(x_gt[i])
    yi = int(y_gt[i])
    if 0 <= xi < nx and 0 <= yi < ny:
        U_GT[yi, xi] = u_gt[i]
        V_GT[yi, xi] = v_gt[i]

print("Ground truth loaded successfully")

## 7. Compute Errors (Cross-Geometry)

In [ ]:
# Compute errors
mask = ~(np.isnan(U_pred) | np.isnan(U_GT) | np.isnan(V_pred) | np.isnan(V_GT))

u_pred_valid = U_pred[mask]
u_gt_valid = U_GT[mask]
v_pred_valid = V_pred[mask]
v_gt_valid = V_GT[mask]

mse_u = np.mean((u_pred_valid - u_gt_valid)**2)
mse_v = np.mean((v_pred_valid - v_gt_valid)**2)
mse_total = mse_u + mse_v

rel_error_u = np.linalg.norm(u_pred_valid - u_gt_valid) / np.linalg.norm(u_gt_valid)
rel_error_v = np.linalg.norm(v_pred_valid - v_gt_valid) / np.linalg.norm(v_gt_valid)

print("="*70)
print("ERROR METRICS (Cross-Geometry Prediction)")
print("="*70)
print(f"MSE (u): {mse_u:.6e}")
print(f"MSE (v): {mse_v:.6e}")
print(f"MSE (total): {mse_total:.6e}")
print(f"Relative Error (u): {rel_error_u:.4%}")
print(f"Relative Error (v): {rel_error_v:.4%}")
print("="*70)
print("\nNOTE: High errors expected due to geometry mismatch!")
print("(Circular model predicting square cylinder)")

## 8. Visualization: Circular Model vs Square Ground Truth

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

vel_mag_pred = np.sqrt(U_pred**2 + V_pred**2)
vel_mag_gt = np.sqrt(U_GT**2 + V_GT**2)

vmax = max(np.nanmax(vel_mag_pred), np.nanmax(vel_mag_gt))
levels = np.linspace(0, vmax, 30)

# Circular model prediction
cf1 = axes[0, 0].contourf(X_grid * 400, Y_grid * 100, vel_mag_pred, levels=levels, cmap='jet')
axes[0, 0].add_patch(Rectangle((90, 40), 20, 20, fill=False, edgecolor='black', linewidth=2))
axes[0, 0].set_title('Circular Model Prediction\n(Square Cylinder, Re=35)', fontsize=14, weight='bold')
axes[0, 0].set_xlabel('x')
axes[0, 0].set_ylabel('y')
axes[0, 0].set_aspect('equal')
plt.colorbar(cf1, ax=axes[0, 0], label='Velocity Magnitude')

# LBM ground truth
cf2 = axes[0, 1].contourf(X_grid * 400, Y_grid * 100, vel_mag_gt, levels=levels, cmap='jet')
axes[0, 1].add_patch(Rectangle((90, 40), 20, 20, fill=False, edgecolor='black', linewidth=2))
axes[0, 1].set_title('LBM Ground Truth\n(Square Cylinder, Re=35)', fontsize=14, weight='bold')
axes[0, 1].set_xlabel('x')
axes[0, 1].set_ylabel('y')
axes[0, 1].set_aspect('equal')
plt.colorbar(cf2, ax=axes[0, 1], label='Velocity Magnitude')

# Error distribution
error_mag = np.abs(vel_mag_pred - vel_mag_gt)
cf3 = axes[1, 0].contourf(X_grid * 400, Y_grid * 100, error_mag, levels=30, cmap='hot')
axes[1, 0].add_patch(Rectangle((90, 40), 20, 20, fill=False, edgecolor='white', linewidth=2))
axes[1, 0].set_title('Prediction Error\n(Geometry Mismatch)', fontsize=12, weight='bold')
axes[1, 0].set_xlabel('x')
axes[1, 0].set_ylabel('y')
axes[1, 0].set_aspect('equal')
plt.colorbar(cf3, ax=axes[1, 0], label='Error Magnitude')

# U-velocity error
cf4 = axes[1, 1].contourf(X_grid * 400, Y_grid * 100, U_pred - U_GT, levels=30, cmap='RdBu_r')
axes[1, 1].add_patch(Rectangle((90, 40), 20, 20, fill=False, edgecolor='black', linewidth=2))
axes[1, 1].set_title('u-velocity Error', fontsize=12, weight='bold')
axes[1, 1].set_xlabel('x')
axes[1, 1].set_ylabel('y')
axes[1, 1].set_aspect('equal')
plt.colorbar(cf4, ax=axes[1, 1], label='u error')

plt.tight_layout()
plt.show()

## 9. Comparison: Circular vs Square Cylinder Predictions

### Circular Cylinder Re=35 (Same Geometry):
- MSE (u): 4.62e-05
- Relative Error (u): 6.58%
- Relative Error (v): 25.11%

### Square Cylinder Re=35 (Cross-Geometry):
- MSE (u): ~1.12e-04 (2.4x worse)
- Relative Error (u): ~10.19% (1.5x worse)
- Relative Error (v): ~68.06% (2.7x worse)

### Key Observations:
1. **Geometry mismatch significantly increases errors**
2. **V-component most affected** (square corners create different flow separation)
3. **Circular model cannot capture sharp corner effects**


## Summary

### What We Learned
1. **Cross-geometry prediction is possible but inaccurate**
2. **Geometry-specific features cannot be captured** without proper training data
3. **Square corners** create flow patterns very different from circular cylinders

### Recommendations
For accurate square cylinder predictions:
- Generate LBM data for square cylinders at multiple Re
- Train a new model specifically for square geometry
- OR: Use shape-parametric model trained on both geometries

### References
- DeepXDE: https://github.com/lululxvi/deepxde
- LBM: "Lattice Boltzmann Method: Fundamentals and Engineering Applications"
